## Softmax回归（多分类）实现

### MNIST数据集的提取

In [1]:
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import My_utils

My_utils.use_svg_display()

In [2]:
trans=transforms.ToTensor()#将image或ndarray转为tensor
train_set=torchvision.datasets.FashionMNIST(root='./data',train=True,transform=trans,download=True)
test_set=torchvision.datasets.FashionMNIST(root='./data',train=False,transform=trans,download=True)

### Fashion-MNIST数据集的处理

In [3]:
def get_fashion_mnist_labels(labels):#索引和label的转换，MNIST数据集是数值的标号
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]#输入的labels是一串数字，表示需要展示的图片的labels，后续i作为该labels的索引

def show_image(imgs,row_num,col_num,titles=None,scale=1.5):
    figsize=(col_num*scale,row_num*scale)
    _,axes=plt.subplot(row_num,col_num,figsize=figsize)
    axes=axes.flatten()
    for i,(ax,img) in enumerate(zip(axes,imgs)):#i是图片的索引
        if torch.is_tensor(img):
            ax.imshow(img.numpy())
        else:
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)#将坐标轴设为不可见
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

### 开始Softmax的实现

#### 准备组件

In [6]:
batch_size=256
train_iter,test_iter=My_utils.load_data_from_fashion_MNIST(batch_size)
#图片是28*28的，也就是784个像素，因此这个网络是一个784->10的网络
num_inputs=784
num_outputs=10
W=torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)#784行，10列
b=torch.zeros(num_outputs,requires_grad=True)

def softmax(X):#对应sigmoid函数，是一个用于映射的函数
    X_exp=torch.exp(X)
    partition=X_exp.sum(1,keepdim=True)#保留列的求和
    return X_exp/partition

def net(X):
    return softmax(torch.matmul(X.reshape(-1,W.shape[0]),W)+b)#shape[0]返回的是他第0维的形状，也就是他的行数；  将X转化为一个256*784的张量

def cross_entropy(y_hat,y):#损失函数
    return -torch.log(y_hat[range(len(y_hat)),y])

def count_accurate(y_hat,y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat=y_hat.argmax(axis=1)
    correct_count=0
    for i in range(len(y_hat)):
        if y_hat[i].type(y.dtype)==y[i]:
            correct_count+=1
    return float(correct_count)

def calc_accuracy(net,data_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()#进入评估模式
    metric=My_utils.Accumulator(2)
    with torch.no_grad():
        for X,y in data_iter:
            metric.add(count_accurate(net(X),y),y.numel())
    return  metric[0]/metric[1]

0.1428

#### 开始训练